In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt


In [ ]:
dir_SLAND = '/Data/SLAND_Trendy-v10_S2_LatLon/'
dir_ctrs  = '/Data/data_ancillary/info_countries/'
dir_grids = '/Data/grids/'
dir_out   = '/Data/SLAND_Trendy-v10_S2_countries/'
dir_tmp   = '/Data/tmp/'


## Calculation

In [ ]:
#Define models
models = ['CABLE-POP', 'CLASSIC', 'CLM5.0', 'DLEM', 'IBIS', 'ISAM', 'ISBA-CTRIP', 'JSBACH', 'JULES-ES-1.1',
          'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs']#'CLASSIC-N', 'ORCHIDEE'

#Read ISO codes for countries and conversions between ISO alpha-3 codes from IPCC and ISO numeric
fname_ISO_num    = dir_ctrs + 'iso_codes_alpha_numeric.xlsx'
fname_IPCC_codes = dir_ctrs + 'IPCC_regions.xlsx'
data_IPCC_codes = pd.read_excel(fname_IPCC_codes, sheet_name='region_classification', header=0, usecols=[0, 1, 3])
data_alph_num   = pd.read_excel(fname_ISO_num, header=0)

#Read ISO codes for countries, IPCC countries, and conversions between ISO alpha-3 codes from IPCC and ISO numeric
fname_ctrs_ISO = dir_ctrs + 'wrld_cntrs_BLUE_TN_upd.nc'
data_ctrs_ISO  = xr.open_dataset(fname_ctrs_ISO)

#Create NetCDF of global land area
fname_landarea_ISO = dir_ctrs + 'wrld_land-area_BLUE_TN_upd.nc'
if os.path.exists(fname_landarea_ISO): os.remove(fname_landarea_ISO)
data_landarea_ISO = 1 * (data_ctrs_ISO.ISOcode>0)
data_landarea_ISO = data_landarea_ISO.to_dataset(name='land_fraction')
data_landarea_ISO.to_netcdf(fname_landarea_ISO)

#Create dicts for storing data
SLAND_coll = dict()

#Loop over models
for model in models:

    print(model)

    #Conservatively regrid global land area to DGVM grid
    fname_landarea_ISO_regr = dir_tmp + 'land-area_cntrs_BLUE_TN_upd_regrid_' + model + '_SLAND_tmp.nc'
    if os.path.exists(fname_landarea_ISO_regr): os.remove(fname_landarea_ISO_regr)
    file_grid = dir_grids + 'grid_xy_' + model
    os.system('cdo remapcon,' + file_grid + ' ' + fname_landarea_ISO + ' ' + fname_landarea_ISO_regr)

    #Read regridded land area file 
    data_landarea_ISO = xr.open_dataset(fname_landarea_ISO_regr)

    #Get file name for SLAND
    fnames = [file for file in os.listdir(dir_SLAND) if (model + '_' in file) and ('SLAND_NBP.nc' in file) and ('NBPPFT' not in file)]
    if len(fnames)!=1:  sys.exit('Filename not unique')

    #Read SLAND data
    fname = dir_SLAND + fnames[0]
    data_SLAND = xr.open_dataset(fname)

    #Get lat and lon names
    if 'latitude' in data_SLAND.dims:  lat_name, lon_name = 'latitude', 'longitude'
    else:                              lat_name, lon_name = 'lat', 'lon'

    #Check that model grid and country grid agree
    check_lat = np.max(np.abs(data_SLAND[lat_name].values - data_landarea_ISO[lat_name].values))
    check_lon = np.max(np.abs(data_SLAND[lon_name].values - data_landarea_ISO[lon_name].values))
    if check_lat>0.01:  sys.exit('Latitudes do not agree')
    if check_lon>0.01:  sys.exit('Longitudes do not agree')

    #Re-index if there are small deviations in lat and lon
    if (check_lat!=0) or (check_lon!=0):
        data_SLAND = data_SLAND.reindex({lat_name: data_landarea_ISO[lat_name], lon_name: data_landarea_ISO[lon_name]}, method='nearest')

    #Create dicts for storing data
    SLAND_ctrs = dict()      

    #Loop over all country codes
    for i, iso_alpha3 in enumerate(data_IPCC_codes['ISO']):

        if np.mod(i, 50)==0:
            print('  -run ' + str(i+1) + ' of ' + str(len(data_IPCC_codes['ISO'])))

        #Get numbeic ISO code of country
        iso_numeric = data_alph_num['Numeric'][data_alph_num['Alpha-3 code']==iso_alpha3].values[0]

        #Select country in country mask
        mask_ISO = 1 * (data_ctrs_ISO.ISOcode==iso_numeric)

        #Define temporary file names for selecting and regridding country
        fname_tmp      = dir_tmp + 'country_fraction_' + iso_alpha3 + '_' + model + '_SLAND_tmp.nc'
        fname_tmp_regr = dir_tmp + 'country_fraction_' + iso_alpha3 + '_' + model + '_regr_SLAND_tmp.nc'
        if os.path.exists(fname_tmp):       os.remove(fname_tmp)
        if os.path.exists(fname_tmp_regr):  os.remove(fname_tmp_regr)

        #Save country fraction file in NetCDF
        mask_ISO = mask_ISO.to_dataset(name='country_fraction')
        mask_ISO.to_netcdf(fname_tmp)

        #Conservatively regrid country fraction to DGVM grid
        file_grid = dir_grids + 'grid_xy_' + model
        os.system('cdo -s remapcon,' + file_grid + ' ' + fname_tmp + ' ' + fname_tmp_regr)

        #Read regridded country fraction file
        mask_ISO_regr = xr.open_dataset(fname_tmp_regr)

        #Perform calculation (if country fraction is not 0 everywhere)
        if len(mask_ISO_regr.data_vars)!=0:

            #Create weighting factor (= fraction of country / land fraction)
            weights = mask_ISO_regr.country_fraction / data_landarea_ISO.land_fraction

            #Get SLAND sum in selected country
            data_sel = (data_SLAND * weights).sum((lat_name, lon_name))

            #Convert to Tg C/year
            data_sel = 1000 * data_sel

            #Save values in dict
            SLAND_ctrs[iso_alpha3] = data_sel.SLAND.values

        else:

            #Set SLAND to 0 if country is too small
            SLAND_ctrs[iso_alpha3] = np.zeros(len(data_SLAND.time))

        #Remove temporary files
        os.remove(fname_tmp)
        os.remove(fname_tmp_regr)

    #Convert data to data frame (and sort by country name)
    SLAND_ctrs_df = pd.DataFrame(SLAND_ctrs, index=data_SLAND.time.dt.year)
    SLAND_ctrs_df = SLAND_ctrs_df.reindex(sorted(SLAND_ctrs_df.columns), axis=1)

    #Add units in first cell
    SLAND_ctrs_df = SLAND_ctrs_df.rename_axis('unit: TG C/year')
    
    #Save data in dictionary
    SLAND_coll[model] = SLAND_ctrs_df
    
    #Define output file name
    fname_out2 = dir_tmp + 'TEST/TMP_SLAND_all_' + model + '.xlsx'
    if os.path.exists(fname_out2): os.remove(fname_out2)
    SLAND_ctrs_df.to_excel(fname_out2, sheet_name=model + '_SLAND_IPCC_ctrs', index=True, header=True, float_format='%.6f')

    #Remove temporary files
    os.remove(fname_landarea_ISO_regr)

#Define output file name
fname_out = dir_out + 'SLAND_S2_DGVMS_countries_ISOcode_corrected.xlsx'

#Create xlsx-file
with pd.ExcelWriter(fname_out) as writer:
    
    #Loop over models
    for model in models:

        #Save as sheet in excel
        SLAND_coll[model].to_excel(writer, sheet_name=model + '_SLAND_IPCC_ctrs', index=True, header=True, float_format='%.6f')
        